In [ ]:
import numpy as np 
import mne 
import matplotlib.pyplot as plt
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from src.GP.gp_algorithms import Multiclass_GP
from src.GP.kernels import FastGA, AGDTW
from dtw import *

In [ ]:
## Simulate two different time series
# First a sine based time series with random offset and phase
N = 100
T = 150
sine_TS = np.zeros((N, T))
x = np.linspace(0, 8*np.pi, T)
sigma = 0.5

for i in range(N):
    phase_shift = np.random.normal(0, np.pi)
    off_set = np.random.normal()
    noise = np.random.normal(0, sigma, T)
    sine_TS[i,:] = np.sin(x+phase_shift) + noise

In [ ]:
## Then an AR(1) time series
AR_TS = np.zeros((N,T))
alpha = np.random.normal()
beta1 = 0.5*np.random.uniform(-1,1)
beta2 = 0.3*np.random.uniform(-1,1)

for i in range(N):
    AR_TS[i,0] = np.random.normal()
    AR_TS[i,1] = alpha + beta1*AR_TS[i,0] + np.random.normal(0, sigma)
    for t in range(2, T):
        noise = np.random.normal(0,sigma)
        AR_TS[i,t] = alpha + beta1*AR_TS[i,t-1] + beta2*AR_TS[i,t-2] + noise

In [ ]:
plt.plot(sine_TS[0,:])
plt.plot(sine_TS[1,:])
plt.show()

plt.plot(AR_TS[0,:])
plt.plot(AR_TS[1,:])
plt.show()

In [ ]:
# Test dynamic time warping
distance_sine, path_sine = fastdtw(sine_TS[0,:],sine_TS[1,:], dist=euclidean)
path_sine = np.array(path_sine)

distance_AR, path_AR = fastdtw(AR_TS[0,:],AR_TS[1,:], dist=euclidean)
path_AR = np.array(path_AR)

In [ ]:
out = dtw(sine_TS[0,:],sine_TS[1,:])
out.index1

In [ ]:
plt.plot(sine_TS[0,:][path_sine[:,0]])
plt.plot(sine_TS[1,:][path_sine[:,1]])
plt.show()

plt.plot(AR_TS[0,:][path_AR[:,0]])
plt.plot(AR_TS[1,:][path_AR[:,1]])
plt.show()

In [ ]:
%%time
x = np.concatenate((AR_TS, sine_TS), axis = 0)
kernel = AGDTW(sigma = 10)
K1 = kernel(x)

In [ ]:
plt.matshow(K1)
plt.show()

In [ ]:
from tslearn.metrics import gak, sigma_gak

class FastGA():
    def __init__(self, sigma):
        self.sigma = sigma
    
    def __call__(self, x, xstar = None):
        if xstar is not None:
            K = np.zeros((len(x), len(xstar)))
            for i in range(len(xstar)):
                print('Observation', i)
                for j in range(len(x)):
                    print('Observation', i)
                    K[j,i] = gak(x[j], xstar[i], sigma=self.sigma)
        else:
            K = np.zeros((len(x),len(x)))
            for i in range(len(x)):
                print('Observation', i)
                for j in range(i, len(x)):
                    print('Observation', i)
                    K[j,i] = gak(x[j], x[i], sigma=self.sigma)
            # add lower triangle
            K = K + K.T - np.diag(K)
            
        return K


In [ ]:

class AGDTW():
    def __init__(self, sigma, **kwargs):
        self.sigma = sigma
    
    def __call__(self, x, xstar = None):
        if xstar is not None:
            K = np.zeros((len(x), len(xstar)))
            for i in range(len(xstar)):
                for j in range(len(x)):
                    out = dtw(xstar[i,:], x[j,:])
                    dist = np.exp(-(xstar[i,out.index1] - \
                                    x[j,out.index2])**2/self.sigma**2)
                    K[j,i] = np.sum(dist)

        else:
            K = np.zeros((len(x), len(x)))
            for i in range(len(x)):
                for j in range(i, len(x)):
                    out = dtw(x[i,:], x[j,:])
                    dist = np.exp(-(x[i,out.index1] - x[j,out.index2])**2/self.sigma**2)
                    K[j,i] = np.sum(dist)
                    
            # add lower triangle
            K = K + K.T - np.diag(K)
        return K